# 실행 가능성 확인하기

In [1]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## 하이퍼파라미터

In [2]:
EPOCHS = 50

DATASET_PATH = 'data/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH = 'results/'

## 단순한 모델 설정

In [3]:
def Model():
    return Sequential([Conv2D(32, (3,3), activation='relu'),
                      MaxPool2D(),
                      Conv2D(64, (3,3), activation='relu'),
                      MaxPool2D(),
                      Conv2D(128, (3,3), activation='relu'),
                      MaxPool2D(),
                      Conv2D(256, (3,3), activation='relu'),
                      MaxPool2D(),
                      Flatten(),
                      Dense(1, activation='sigmoid')] # binary clf니까 sigmoid 사용
                     )

## 데이터셋 불러오기

In [4]:
# file_name을 불러오고 png로 저장되어있기때문에 256x256로 풀어주는 디코딩하는 작업
def preprocess(file_name):
    img = tf.io.read_file(file_name) # 내용을 binary로 읽어옴
    img = tf.image.decode_png(img) # png는 channel입력 안 받음(image는 받음)
    return tf.image.convert_image_dtype(img, tf.float32)
    # 읽어올 때는 uint8으로 읽어오는데 그것을 float32형태로 변환해서 return

In [6]:
ok_list = glob.glob(DATASET_OK_PATTERN)
#print(ok_list)
# glob사용하면 pattern으로 되어있는 파일들을 쭉 읽어서 리스트 형태로 파일 저장
# os.listdir()을 사용하면 'data/2/OK'가 빠진 상태로 저장되고, 파일 이름 뒤에 '.png'도 지정해줄 수 없어서 불편함

ds_ok = tf.data.Dataset.list_files(ok_list)     # 양품 : 0
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list)) # 0/1 label중 어떤 label인지 설정
# 또는 [0 for _ in range(len(ok_list))]

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label)) # zip -> 이미지만 출력했다면 label도 같이 출력하게끔


fail_list = glob.glob(DATASET_FAIL_PATTERN)

ds_fail = tf.data.Dataset.list_files(fail_list)   # 불량 : 1
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train, Valid 데이터셋 나누기

In [7]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size) # 얼만큼을 모아서 shufflt해줄 것이냐 -> 전체 데이터를 다 섞자
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32) # train_size 크기만큼 가져와서 1024개의 버퍼사이즈를 가지고 매 epoch 돌때마다 shuffle
ds_valid = ds.skip(train_size).batch(32) # take대신 skip -> 그만큼 skip해줌

## 모델 생성 및 학습

In [8]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Train for 21 steps, validate for 9 steps
Epoch 1/50
21/21 [==============================] - 112s 5s/step - loss: 0.4154 - accuracy: 0.8427 - val_loss: 0.4102 - val_accuracy: 0.8556
Epoch 2/50
21/21 [==============================] - 135s 6s/step - loss: 0.4215 - accuracy: 0.8669 - val_loss: 0.3883 - val_accuracy: 0.8768
Epoch 3/50
21/21 [==============================] - 125s 6s/step - loss: 0.3765 - accuracy: 0.8880 - val_loss: 0.4336 - val_accuracy: 0.8521
Epoch 4/50
21/21 [==============================] - 82s 4s/step - loss: 0.3665 - accuracy: 0.8729 - val_loss: 0.3373 - val_accuracy: 0.8697
Epoch 5/50
21/21 [==============================] - 101s 5s/step - loss: 0.3428 - accuracy: 0.8805 - val_loss: 0.3842 - val_accuracy: 0.8627
Epoch 6/50
21/21 [==============================] - 82s 4s/step - loss: 0.3196 - accuracy: 0.8880 - val_loss: 0.3561 - val_accuracy: 0.8803
Epoch 7/50
21/21 [==============================] - 91s 4s/step - loss: 0.3627 - accuracy: 0.8684 - val_loss: 0.390

## 결과를 이미지로 저장

In [10]:
# 결과를 잘 볼 수 있어야 분석할 수 있겠지

# 경로 저장
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)
        
mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP') # 실제 불량
mkdir(RESULT_SAVE_PATH + '/TN') # 실제 양품
mkdir(RESULT_SAVE_PATH + '/FP') # 가짜 불량 (양품인데 불량이라고 분류)
mkdir(RESULT_SAVE_PATH + '/FN') # 가짜 양품 (불량인데 양품이라고 분류)

# validation data을 돌면서 결과 저장
index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]): # 32개의 batch size로 돌기때문에 한번에 데이터가 32개가 나오니까 하나씩 사용해주기위해
        gt = labels[idx].numpy() # labels은 tensor로 저장해줬기때문에 numpy로 바꿔줘야 파이썬 문법 사용하기 쉽겠지
        y = preds[idx]
            
        # 1 : 불량
        if gt == 1 and y > 0.5 : # binary clf는 0.5를 기준으로 하게끔 되어있음
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5 :
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5 :
            path = RESULT_SAVE_PATH + '/FP'
        else :
            path = RESULT_SAVE_PATH + '/TN'
            
        cv2.imwrite(path + '/%.f_%04d.png' % (y, index), imgs[idx].numpy() * 255) # img에 255를 곱해줘야 cv에서 이미지 저장됨  
        index +=1